In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
import hashlib

def generate_document_id(doc):
    combined= f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]

    return document_id

In [4]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [5]:
documents[3]

{'text': "You don't need it. You're accepted. You can also just start learning and submitting homework without registering. It is not checked against any registered list. Registration is just to gauge interest before the start date.",
 'section': 'General course-related questions',
 'question': 'Course - I have registered for the Data Engineering Bootcamp. When can I expect to receive the confirmation email?',
 'course': 'data-engineering-zoomcamp',
 'id': '0bbf41ec'}

In [5]:
from collections import defaultdict

In [6]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [10]:
len(hashes), len(documents) # should be the same if all ids are unique

(947, 948)

In [11]:
for k, values in hashes.items():
    if(len(values) > 1):
        print(k, len(values))

593f7569 2


In [13]:
hashes['593f7569'] # duplicate question

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [7]:
import json
with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [12]:
prompt_template = """
You emulate a student who's taking our course.
Formulate 5 questions this student might ask based on a FAQ record. The record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible for the record.

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:
["question1", "question2", ..., "question5"]

""".strip()

In [10]:
from google import genai
import os

GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

client = genai.Client(api_key=GEMINI_API_KEY)

In [14]:
doc = documents[2]
prompt = prompt_template.format(**doc)

response = client.models.generate_content(
        model="gemini-2.0-flash",
        # config=types.GenerateContentConfig(
            # system_instruction="You are a cat. Your name is Neko."),
        contents=prompt
    )

In [37]:
# json.loads(response.candidates[0].content.parts[0].text)
# response.candidates[0].content.parts[0]
test = response.text.replace("json", "").replace("```",  "", 2)
test

'\n[\n  "If I don\'t register for the course officially, am I still allowed to submit the homework assignments?",\n  "Is it possible to submit homework assignments even if I join the course late?",\n  "Are there any specific deadlines that I should be aware of, especially concerning the final projects in this course?",\n  "Even though I can join late and submit homework, does the flexibility extend to final project deadlines, or are they strict?",\n  "What happens if I start the course after the official start date? Will I still be able to participate fully and submit all assignments including the final project?"\n]\n'

In [38]:
json.loads(test)

["If I don't register for the course officially, am I still allowed to submit the homework assignments?",
 'Is it possible to submit homework assignments even if I join the course late?',
 'Are there any specific deadlines that I should be aware of, especially concerning the final projects in this course?',
 'Even though I can join late and submit homework, does the flexibility extend to final project deadlines, or are they strict?',
 'What happens if I start the course after the official start date? Will I still be able to participate fully and submit all assignments including the final project?']

In [39]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        # config=types.GenerateContentConfig(
            # system_instruction="You are a cat. Your name is Neko."),
        contents=prompt
    )

    json_string_gemini = response.candidates[0].content.parts[0].text

    return json_string_gemini.replace("json", "").replace("```",  "", 2)

In [40]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
results  = {}


In [43]:

for doc in tqdm(documents):
    doc_id = doc['id']
    # pseudo-cache in case of failure
    if doc_id in results:
        continue
    questions = generate_questions(doc)
    results[doc_id] = questions

100%|█████████████████████████████████████████████████████████████████████| 948/948 [32:24<00:00,  2.05s/it]


In [45]:
import pickle

In [47]:
with open('results.bin', 'wb') as f_in:
    pickle.dump(results, f_in)

In [78]:
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

JSONDecodeError: Expecting value: line 8 column 1 (char 1011)

In [81]:
json_questions=[
r'\n[\n  "I'm looking to deploy my BentoML model as an Amazon Lambda function. Is there a recommended approach or specific guidance available for passing the BentoML content or Docker container to Lambda?",\n  "I\'ve heard that deploying BentoML models to serverless environments like AWS Lambda can be challenging. Does BentoML provide resources or tutorials specifically addressing this use case?",\n  "I need to understand the process of packaging and deploying a BentoML application, potentially as a Docker container, to Amazon Lambda. Is there a good resource I can use to learn how to do this?",\n  "When deploying a BentoML model on AWS Lambda, what are the key considerations for ensuring compatibility and optimal performance? Is there a video tutorial demonstrating a recommended deployment strategy?",\n  "Are there any practical demonstrations or walkthroughs available that illustrate how to deploy a BentoML model to Amazon Lambda using a Docker container, and how to pass the necessary configurations?",\n]\n'
]

SyntaxError: '[' was never closed (3258507300.py, line 1)

In [70]:
json.dumps(json_questions)

'["\\\\n[\\\\n  \\"Why am I getting a TypeError when trying to set `feature_names` in `xgb.DMatrix`, and the error message says \\\\\\\\\\"Expecting a sequence of strings for feature names, got: <class \\\\\'numpy.ndarray\\\\\'>\\\\\\\\\\"?\\",\\\\n  \\"I\\\\\'m encountering a ValueError with the message \\\\\\\\\\"feature_names must be string, and may not contain [, ] or <\\\\\\\\\\" when using `xgb.DMatrix`. How can I resolve this error, even after converting the feature names to a list?\\",\\\\n  \\"The FAQ mentions that `dv.feature_names_` returns feature names with characters XGBoost doesn\\\\\'t like. Could you explain more clearly what characters cause problems for XGBoost and why?\\",\\\\n  \\"If I am getting errors related to `feature_names` in `xgb.DMatrix` due to characters like \\\\\'[\\\\\', \\\\\']\\\\\', or \\\\\'<\\\\\', what are the recommended solutions besides completely omitting the `feature_names=` argument when creating the DMatrix?\\",\\\\n  \\"The FAQ suggests u

In [76]:
doc_id

'15f361b7'

In [77]:
results[doc_id] = json.dumps(json_questions)